# Exploratory Data Analysis

## The Immigration Data

In [1]:
import pandas as pd

In [13]:
fname = 'data/18-83510-I94-Data-2016/i94_jan16_sub.sas7bdat'

In [14]:
myiter = pd.read_sas(fname, 'sas7bdat', encoding='ISO-8859-1', chunksize=20)

In [15]:
for df in myiter:
    break
    
df.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,7.0,2016.0,1.0,101.0,101.0,BOS,20465.0,1.0,MA,NaN,...,NaN,NaN,1996.0,D/S,M,NaN,LH,346608285.0,424,F1
1,8.0,2016.0,1.0,101.0,101.0,BOS,20465.0,1.0,MA,NaN,...,NaN,NaN,1996.0,D/S,M,NaN,LH,346627585.0,424,F1
2,9.0,2016.0,1.0,101.0,101.0,BOS,20469.0,1.0,CT,20480.0,...,NaN,M,1999.0,07152016,F,NaN,AF,381092385.0,338,B2
3,10.0,2016.0,1.0,101.0,101.0,BOS,20469.0,1.0,CT,20499.0,...,NaN,M,1971.0,07152016,F,NaN,AF,381087885.0,338,B2
4,11.0,2016.0,1.0,101.0,101.0,BOS,20469.0,1.0,CT,20499.0,...,NaN,M,2004.0,07152016,M,NaN,AF,381078685.0,338,B2


In [16]:
list(df.columns.values)

['cicid',
 'i94yr',
 'i94mon',
 'i94cit',
 'i94res',
 'i94port',
 'arrdate',
 'i94mode',
 'i94addr',
 'depdate',
 'i94bir',
 'i94visa',
 'count',
 'dtadfile',
 'visapost',
 'occup',
 'entdepa',
 'entdepd',
 'entdepu',
 'matflag',
 'biryear',
 'dtaddto',
 'gender',
 'insnum',
 'airline',
 'admnum',
 'fltno',
 'visatype']

## Immigration Data Headers File

`data/I94_SAS_Labels_Descriptions.SAS` is a text file with descriptions of what each column is and what can go inside it. Will be a bit of work to parse it, but should help